In [ ]:
import pickle
import numpy as np
import heapq
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

folder_path = 'vectores'

archivos_pkl = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]

embeddings = []
image_names = []

for archivo in archivos_pkl:
    archivo_pkl = os.path.join(folder_path, archivo)
    
    with open(archivo_pkl, 'rb') as f:
        datos = pickle.load(f)
    
    for item in datos:
        if 'embedding' in item and item['embedding'] is not None and len(item['embedding']) > 0:
            embeddings.append(item['embedding'][0])  
            image_names.append(item['image_name'])  

def cosine_similarity(v1, v2):
    """Calcula la similitud de coseno entre dos vectores."""
    dot_product = np.dot(v1, v2)  
    norm_v1 = np.linalg.norm(v1)   
    norm_v2 = np.linalg.norm(v2)   
    return dot_product / (norm_v1 * norm_v2)

def KnnSearchCoseno(data, query, k):
    max_heap = []
    
    for i in range(len(data)):
        sim = cosine_similarity(data[i], query)  
        
        if len(max_heap) < k:
            heapq.heappush(max_heap, (sim, i))  
        else:
            heapq.heappushpop(max_heap, (sim, i))  
    
    return [(image_names[i], sim) for sim, i in max_heap]  





In [ ]:
images_df = pd.read_csv('images.csv/images.csv')

query_index = 6
query = embeddings[query_index]  
imagen_q_name = image_names[query_index]

imagen_q = images_df[images_df['filename'] == imagen_q_name].iloc[0]

print("La imagen de la query es: ", imagen_q)

image_link = imagen_q['link']
print("Link de la imagen:", image_link)

# Realizar la búsqueda KNN usando similitud de coseno
k = 5
resultados = KnnSearchCoseno(embeddings, query, k)

print(f"Las {k} imágenes más cercanas a {image_names[query_index]} son:")

for img_name, sim in resultados:
    print(f"Imagen: {img_name}, Similitud: {sim}")
    image_row = images_df[images_df['filename'] == img_name]
    if not image_row.empty:
        print("Link de la imagen:", image_row['link'].values[0])

La imagen de la query es:  filename                                            30805.jpg
link        http://assets.myntassets.com/v1/images/style/p...
Name: 6, dtype: object
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/06e9d4231254fdb2c7fe967ad413ad7b_images.jpg
Las 5 imágenes más cercanas a 30805.jpg son:
Imagen: 41587.jpg, Similitud: 0.930131493442379
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/036f2c734abc53620813580f530e4cdb_images.jpg
Imagen: 14669.jpg, Similitud: 0.9326281617052574
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/3aa18746c06db000ce9177998ef34f2b_images.jpg
Imagen: 30805.jpg, Similitud: 1.0000000000000002
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/06e9d4231254fdb2c7fe967ad413ad7b_images.jpg
Imagen: 23199.jpg, Similitud: 0.9512444918589128
Link de la imagen: http://assets.myntassets.com/v1/images/style/properties/b4d30abb75c82962a03e9ae936fd63ff_im